In [124]:
import numpy as np
import pandas as pd

In [125]:
credit = pd.read_csv('credit.csv')
movies = pd.read_csv('movies.csv')

In [126]:
movies = movies.merge(credit, on='title')

In [127]:
movies.to_json('movies.json', orient='records', lines=True)

In [128]:
movies = movies[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew' ]]

In [129]:
# movies['crew'][0]
import ast
def director(data):
    dire = []
    for i in ast.literal_eval(data):
        if i['job'] == 'Director':
            dire.append(i['name'])
            break
    return dire

In [130]:
movies['crew'] = movies['crew'].apply(director)

In [131]:
movies.dropna(inplace=True)

In [132]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [133]:
movies['genres'] = movies['genres'].apply(convert)

In [134]:
movies['cast'] = movies['cast'].apply(convert)

In [135]:
movies['keywords'] = movies['keywords'].apply(convert)

In [136]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [137]:
movies['cast'] = movies['cast'].apply(lambda x:x[:3])

In [138]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['overview'] = movies['overview'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])

In [139]:
movies['tags'] = movies['overview']+ movies['genres'] +  movies['keywords'] + movies['cast'] + movies['crew']

In [140]:
newMovies = movies[['id', 'title', 'tags']]

In [141]:
newMovies['tags'] = newMovies['tags'].apply(lambda x:' '.join(x))

C:\Users\ANKIT SHARMA\AppData\Local\Temp\ipykernel_2536\18303954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newMovies['tags'] = newMovies['tags'].apply(lambda x:' '.join(x))


In [142]:
newMovies['tags'] = newMovies['tags'].apply(lambda x:x.lower())

C:\Users\ANKIT SHARMA\AppData\Local\Temp\ipykernel_2536\1981169705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newMovies['tags'] = newMovies['tags'].apply(lambda x:x.lower())


In [143]:
newMovies['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [145]:
vectors = cv.fit_transform(newMovies['tags']).toarray()

In [146]:
import nltk

In [147]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [148]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)
        

In [149]:
newMovies['tags'] = newMovies['tags'].apply(stem)

C:\Users\ANKIT SHARMA\AppData\Local\Temp\ipykernel_2536\601366695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newMovies['tags'] = newMovies['tags'].apply(stem)


In [150]:
from sklearn.metrics.pairwise import cosine_similarity

In [151]:
similarity = cosine_similarity(vectors)

In [152]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]

[(539, 0.26089696604360174),
 (1192, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1214, 0.24944382578492943)]

In [153]:
def recommended(movie):
    movie= movie.title()
    moviesIndex = newMovies[newMovies['title']==movie].index[0]
    distance = similarity[moviesIndex]
    movies_list = sorted(list(enumerate(distance)), reverse = True, key = lambda x:x[1])[1:11]
    for i in movies_list:
        print(newMovies.iloc[i[0]].title)
    

In [154]:
recommended('Inception')

Duplex
The Helix... Loaded
Star Trek II: The Wrath of Khan
Timecop
Chicago Overcoat
Looper
Premium Rush
Transformers: Revenge of the Fallen
Congo
Flatliners


In [ ]:
import numpy as np
import pandas as pd
credit = pd.read_csv('credit.csv')
movies = pd.read_csv('movies.csv')
movies = movies.merge(credit, on='title')
movies.to_json('movies.json', orient='records', lines=True)
movies = movies[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew' ]]
import ast
def director(data):
    dire = []
    for i in ast.literal_eval(data):
        if i['job'] == 'Director':
            dire.append(i['name'])
            break
    return dire
movies['crew'] = movies['crew'].apply(director)
movies.dropna(inplace=True)
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
movies['genres'] = movies['genres'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['cast'] = movies['cast'].apply(lambda x:x[:3])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['overview'] = movies['overview'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['tags'] = movies['overview']+ movies['genres'] +  movies['keywords'] + movies['cast'] + movies['crew']
newMovies = movies[['id', 'title', 'tags']]
newMovies['tags'] = newMovies['tags'].apply(lambda x:' '.join(x))
newMovies['tags'] = newMovies['tags'].apply(lambda x:x.lower())
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

vectors = cv.fit_transform(newMovies['tags']).toarray()
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

newMovies['tags'] = newMovies['tags'].apply(stem)
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]
def recommended(movie):
    movie= movie.title()
    moviesIndex = newMovies[newMovies['title']==movie].index[0]
    distance = similarity[moviesIndex]
    movies_list = sorted(list(enumerate(distance)), reverse = True, key = lambda x:x[1])[1:11]
    for i in movies_list:
        print(newMovies.iloc[i[0]].title)
    
recommended('Inception')